In [1]:
import sys
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"
d = os.path.dirname(os.getcwd())
sys.path.insert(0, d)

In [2]:
from datetime import datetime as dt
import torch
print(torch.__version__)
print(torch.cuda.is_available())

import torch.nn as nn
import torch.nn.functional as F
from torch import optim

import torchvision
from torchvision.datasets import CIFAR10, MNIST
from torchvision.transforms import transforms

from torch.utils.data import DataLoader

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from utils.attacks import pgd
from utils.utils import plot_images
from models.mnist_model import *

1.7.0
True


In [3]:
from datetime import datetime as dt

def clean_acc(model, loader):
    clean_correct = 0.
    model.eval()
    with torch.no_grad():
        for k, (X, y) in enumerate(loader, 1):
            start = dt.now().replace(microsecond=0)
            X, y = X.to(device), y.to(device)  
            clean_correct += (model(X).argmax(dim=1) == y).sum().item()
            end = dt.now().replace(microsecond=0)
            print(f"iter: {k}/{len(loader)} time: {end-start}", end="\r")
        print(" " * 50, end="\r")
        acc = round(clean_correct / len(loader.dataset), 4)
        print(f"clean accuracy: {acc}")
    return acc

def adv_acc(model, loader, loss_fn, attack, attack_params):
    model.eval()
    adv_correct = [0] * len(attack_params)
    for i, (name, params) in enumerate(attack_params):
        for k, (X, y) in enumerate(loader, 1):
            start = dt.now().replace(microsecond=0)
            X, y = X.to(device), y.to(device)
            noise = attack(model, X, y, loss_fn, **params) 
            adv_correct[i] += (model(X+noise).argmax(dim=1) == y).sum().item()
            end = dt.now().replace(microsecond=0)
            print(f"iter: {k}/{len(loader)} time: {end-start}", end="\r")
        print(" " * 50, end="\r")
        print(f"name: {name} accuracy: {(adv_correct[i] / len(loader.dataset)):.4f}")
    return [round(a/len(loader.dataset), 4) for a in adv_correct]


def print_table(models, attacks, clean, adv):
    d = {'model': models, 'clean image': clean}
    for i, (name, p) in enumerate(attacks):
        d[name] = [adv[j][i] for j, _ in enumerate(adv)]

    return pd.DataFrame(data=d)

In [4]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cuda


In [5]:
test_transform = transforms.Compose([
    transforms.ToTensor(),
])

ds = MNIST('../data', train=False, target_transform=None, download=True, transform=test_transform)
test_loader = DataLoader(ds, 2000, shuffle=True)

In [6]:
def create_attack(eps, alpha, num_iter):
    d = {
        "epsilon": eps,
        "alpha": alpha,
        "num_iter": num_iter
    }
    return (f"e={eps:.1f},a={alpha:.1f},i={num_iter}", d)
    

attack1 = create_attack(eps=0.1, alpha=0.1, num_iter=30)
attack2 = create_attack(eps=0.2, alpha=0.1, num_iter=30)
attack3 = create_attack(eps=0.3, alpha=0.1, num_iter=30)


params = [attack1, attack2, attack3]

In [7]:
dtl_clean, dtl_adv = [], []

In [8]:
model = MnistBaseModel().to(device)
model.load_state_dict(torch.load("../saved/mnist_base_model.pt", map_location=device))

acc = clean_acc(model, test_loader)
acc_adv = adv_acc(model, test_loader, nn.CrossEntropyLoss(), pgd, params)

dtl_clean.append(acc)
dtl_adv.append(acc_adv)

clean accuracy: 0.9677                            
name: e=0.1,a=0.1,i=30 accuracy: 0.0204           
name: e=0.2,a=0.1,i=30 accuracy: 0.0000           
name: e=0.3,a=0.1,i=30 accuracy: 0.0000           


In [9]:
model = MnistDTLModel().to(device)
model.load_state_dict(torch.load("../saved/mnist_dtl_model.pt", map_location=device))

acc = clean_acc(model, test_loader)
acc_adv = adv_acc(model, test_loader, nn.CrossEntropyLoss(), pgd, params)

dtl_clean.append(acc)
dtl_adv.append(acc_adv)

clean accuracy: 0.7788                            
name: e=0.1,a=0.1,i=30 accuracy: 0.1117           
name: e=0.2,a=0.1,i=30 accuracy: 0.0862           
name: e=0.3,a=0.1,i=30 accuracy: 0.0837           


In [10]:
models_names = ["Base Model", "DTL Model"]
dtl_results = print_table(models_names, params, dtl_clean, dtl_adv)
dtl_results

,model,clean image,"e=0.1,a=0.1,i=30","e=0.2,a=0.1,i=30","e=0.3,a=0.1,i=30"
0,Base Model,0.9677,0.0204,0.0000,0.0000
1,DTL Model,0.7788,0.1117,0.0862,0.0837


In [11]:
dtl_conv_clean, dtl_conv_adv = [], []

In [12]:
model = MnistBaseConvModel().to(device)
model.load_state_dict(torch.load("../saved/mnist_base_conv_model.pt", map_location=device))

acc = clean_acc(model, test_loader)
acc_adv = adv_acc(model, test_loader, nn.CrossEntropyLoss(), pgd, params)

dtl_conv_clean.append(acc)
dtl_conv_adv.append(acc_adv)

clean accuracy: 0.9855                            
name: e=0.1,a=0.1,i=30 accuracy: 0.3660           
name: e=0.2,a=0.1,i=30 accuracy: 0.0032           
name: e=0.3,a=0.1,i=30 accuracy: 0.0000           


In [13]:
model = MnistDTLConvModel().to(device)
model.load_state_dict(torch.load("../saved/mnist_dtl_conv_model.pt", map_location=device))

acc = clean_acc(model, test_loader)
acc_adv = adv_acc(model, test_loader, nn.CrossEntropyLoss(), pgd, params)

dtl_conv_clean.append(acc)
dtl_conv_adv.append(acc_adv)

clean accuracy: 0.8896                            
name: e=0.1,a=0.1,i=30 accuracy: 0.4869           
name: e=0.2,a=0.1,i=30 accuracy: 0.1599           
name: e=0.3,a=0.1,i=30 accuracy: 0.1388           


In [14]:
model = MnistBaseConvModel().to(device)
model.load_state_dict(torch.load("../saved/mnist_base_conv_model_AT.pt", map_location=device))

acc = clean_acc(model, test_loader)
acc_adv = adv_acc(model, test_loader, nn.CrossEntropyLoss(), pgd, params)

dtl_conv_clean.append(acc)
dtl_conv_adv.append(acc_adv)

clean accuracy: 0.9775                            
name: e=0.1,a=0.1,i=30 accuracy: 0.9629           
name: e=0.2,a=0.1,i=30 accuracy: 0.9383           
name: e=0.3,a=0.1,i=30 accuracy: 0.8607           


In [15]:
model = MnistDTLConvModel().to(device)
model.load_state_dict(torch.load("../saved/mnist_dtl_conv_model_AT.pt", map_location=device))

acc = clean_acc(model, test_loader)
acc_adv = adv_acc(model, test_loader, nn.CrossEntropyLoss(), pgd, params)

dtl_conv_clean.append(acc)
dtl_conv_adv.append(acc_adv)

clean accuracy: 0.8881                            
name: e=0.1,a=0.1,i=30 accuracy: 0.8735           
name: e=0.2,a=0.1,i=30 accuracy: 0.8517           
name: e=0.3,a=0.1,i=30 accuracy: 0.7931           


In [16]:
models_names = ["Base Conv Model", "DTL Conv Model", "Base Conv Model AT", "DTL Conv Model AT"]
dtl_conv_results = print_table(models_names, params, dtl_conv_clean, dtl_conv_adv)
dtl_conv_results

,model,clean image,"e=0.1,a=0.1,i=30","e=0.2,a=0.1,i=30","e=0.3,a=0.1,i=30"
0,Base Conv Model,0.9855,0.3660,0.0032,0.0000
1,DTL Conv Model,0.8896,0.4869,0.1599,0.1388
2,Base Conv Model AT,0.9775,0.9629,0.9383,0.8607
3,DTL Conv Model AT,0.8881,0.8735,0.8517,0.7931
